In [178]:
from math import sqrt,log
import random
import numpy as np
from numpy.linalg import norm
from scipy.special import lambertw
from os import listdir
from os.path import isfile, join
import re
from operator import itemgetter
from collections import Counter
from itertools import combinations

## Read documents

In [19]:
input_dir = "./docs"
onlyfiles = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
docs = []
for fname in onlyfiles:
    with open(join(input_dir, fname), "r") as file:
        docs += [file.read()]

## Clean documents (optional)

In [20]:
docs = [re.sub('\W+', ' ', doc) for doc in docs]

## Compute shingles' hash values

In [21]:
class Shingling():
    def __init__(self, k):
        self.k = k
        self.hash = hash # Use python built-in hashing function
    def transform(self, doc):
        # Compute shingles
        shingles = np.array([doc[i:i+self.k] for i in range(0, len(doc) - self.k + 1)])
        # Filter out duplicates and sort
        hashes = sorted(set([self.hash(shingle) for shingle in shingles]))
        
        return hashes

In [22]:
sh = Shingling(9)
sets = [sh.transform(doc) for doc in docs]

## Compare shingle hashes with Jaccard similarity

In [23]:
def compare_shingles(a, b):
    """Returns the jaccard similarity between two sets.
        a -> set
        b -> set
    """
    return len(set(a) & set(b)) / len(set(a) | set(b))

In [24]:
similarities = {(onlyfiles[i],onlyfiles[j]): compare_shingles(sets[i], sets[j]) 
                for i in range(0, len(docs)) 
                for j in range(i+1, len(docs))}
# Show similarities greater than a threshold
sorted([(k,v) for k,v in similarities.items() if v > 0.4], key=itemgetter(1), reverse=True)

[(('B2.txt', 'B1.txt'), 1.0),
 (('A1small.txt', 'A1.txt'), 0.9586281981491562),
 (('B1small.txt', 'B1.txt'), 0.45179335307666996),
 (('B1small.txt', 'B2.txt'), 0.45179335307666996)]

## Compute minHash signatures

In [25]:
class MinHashing():
    def __init__(self, k):
        self.k = k
        self.c = 4294967311 # Big prime
        self.a_coeffs = random.sample(range(1, 2**31), k)
        self.b_coeffs = random.sample(range(1, 2**31), k)
        
    def _hash(self, x, i):
        return (self.a_coeffs[i]*x + self.b_coeffs[i])%self.c
        
    def transform(self, shingles):
        signatures = np.array([[self._hash(shingle, i) for shingle in shingles] for i in range(self.k)])
        return np.array([l[np.argmin(l)] for l in signatures])


In [276]:
mh = MinHashing(100)
signatures = [mh.transform(s) for s in sets]

## Compare minHash signatures

In [27]:
def compare_signatures(a, b):
    """Computes the similarity between 2 signatures.
        a: numpy array
        b: numpy array
    """
    if len(a) != len(b):
        raise ValueError("Signatures lengths differ.")
    return np.mean(a == b)

In [28]:
estimations = {(onlyfiles[i],onlyfiles[j]):compare_signatures(signatures[i], signatures[j]) 
                for i in range(0, len(signatures)) 
                for j in range(i+1, len(signatures))}
# Show similarity estimations greater than a threshold
sorted([(k,v) for k,v in estimations.items() if v > 0.4], key=itemgetter(1), reverse=True)

[(('B2.txt', 'B1.txt'), 1.0), (('A1small.txt', 'A1.txt'), 0.93999999999999995)]

## Compare minHash estimations against real similarities

In [29]:
errors = {(onlyfiles[i],onlyfiles[j]):abs(estimations[(onlyfiles[i],onlyfiles[j])] - similarities[(onlyfiles[i],onlyfiles[j])])
           for i in range(0, len(docs)) 
           for j in range(i+1, len(docs))}
# Show errors greater than a threshold (e.g. 5%)
sorted([(k,v) for k,v in errors.items() if v > 0.05], key=itemgetter(1), reverse=True)

[(('A1.txt', 'a0200021.txt'), 0.08523326572008115),
 (('B1.txt', 'a0200014.txt'), 0.084402895054282279),
 (('B2.txt', 'a0200014.txt'), 0.084402895054282279),
 (('a0200045.txt', 'a0200021.txt'), 0.082571428571428587),
 (('B1small.txt', 'B1.txt'), 0.081793353076669961),
 (('B1small.txt', 'B2.txt'), 0.081793353076669961),
 (('A1small.txt', 'a0200021.txt'), 0.081456310679611649),
 (('a0200047.txt', 'a0200014.txt'), 0.079720670391061454),
 (('B2.txt', 'a0200021.txt'), 0.066504065040650415),
 (('B1.txt', 'a0200021.txt'), 0.066504065040650415),
 (('A1small.txt', 'a0200047.txt'), 0.057794994040524433),
 (('A1.txt', 'a0200014.txt'), 0.056637841577308903),
 (('A1small.txt', 'a0200014.txt'), 0.053564635435012331),
 (('a0200060.txt', 'a0200021.txt'), 0.053046184081231573),
 (('a0200029.txt', 'a0200047.txt'), 0.053036437246963566),
 (('A1.txt', 'a0200047.txt'), 0.053032015065913374),
 (('a0200021.txt', 'a0200047.txt'), 0.050838677849037978),
 (('A1.txt', 'a0200063.txt'), 0.050621733645701932),
 (('

## LSH: Locality-Sensitive Hashing

20


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:4: ComplexWarning: Casting complex values to real discards the imaginary part


In [305]:
class LSH:
    def __init__(self, matrix, t):
        self.t = t
        self.M = matrix
        self.k = self.M.shape[0]
        self.r = int(float(-lambertw(-self.k*log(t))/log(t)))
        self.b = int(self.k/self.r)

    def index(self):
        band_size = self.b+1 if self.b*self.r!=self.k else self.b
        hashes = [[norm(self.M[i*self.r:(i+1)*self.r, j]) for j in range(self.M.shape[1])] for i in range(0, band_size)]
        #print([[self.M[i:(i+1)*self.r, j] for j in range(self.M.shape[1])] for i in range(0, band_size)])
        self.hashtables = []
        for j,band in enumerate(hashes):
            self.hashtables += [{}]
            for i,h in enumerate(band):
                if h not in self.hashtables[j]:
                    self.hashtables[j][h] = []
                self.hashtables[j][h] += [i]
        #print(self.hashtables)

    def get_pairs(self):
        if not hasattr(self, 'hashtables'):
            raise Error("Call index first")
        pairs = []
        for table in self.hashtables:
            for bucket in table.values():
                pairs += combinations(bucket, 2)
        print(Counter(pairs))
        return [key for key,value in Counter(pairs).items() if value >= self.t*self.b]

In [309]:
lsh = LSH(np.array(signatures).T, 0.4)
print(lsh.k)
print(lsh.b)
print(lsh.r)
lsh.index()
print(lsh.get_pairs())

100
33
3
Counter({(7, 9): 34, (1, 4): 29, (6, 9): 3, (6, 7): 3, (2, 7): 1, (5, 9): 1, (12, 14): 1, (2, 9): 1, (7, 12): 1, (5, 12): 1, (5, 7): 1, (2, 12): 1, (2, 5): 1, (9, 12): 1})
[(1, 4), (7, 9)]


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
